In [1]:
#importing necessary packages
import string
import re
import math
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
import nltk, re, pprint
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import words
from nltk.util import ngrams
import random
from nltk.probability import FreqDist
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tifat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tifat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_words_list(text_name):
    words = nltk.word_tokenize(text_name.lower())
    words = [word for word in words if word.isalpha()]
    
    freq_dist = nltk.FreqDist(words)
    
    return freq_dist

In [3]:
def unigram_probability(words):
    
    unigram_prob = {}
    total_words = sum(words.values())
    
    for key, value in words.items():
        uni_prob = value / total_words
        unigram_prob.update({key:uni_prob})
    
    return unigram_prob

In [4]:
def bigram_probability(b_freq, p_w):
    bigram_prob = {}
    
    for key, value in b_freq.items():
        if value > 1:
            prob = value / p_w[key[0]]
            bigram_prob.update({key:prob})
    
    return bigram_prob
                

In [5]:
def trigram_probability(t_freq, b_freq):
    trigram_prob = {}
    
    for key, value in t_freq.items():
        if value > 1:
            for k,v in b_freq.items():
                if k[0]==key[0] and k[1]==key[1]:
                    prob = value / v
                    trigram_prob.update({key:prob})
    return trigram_prob
    

In [6]:
open_file = open('data/train.txt', 'r', encoding='utf-8')
train1 = open_file.read()

train_words_list = get_words_list(train1)
unigram_prob = unigram_probability(train_words_list)


In [18]:
words = nltk.word_tokenize(train1.lower())
words = [word for word in words if word.isalpha()]

bigrm = nltk.bigrams(words)
b_freq = nltk.FreqDist(bigrm)
b_freq_y = {}
for k,v in b_freq.items():
    name = "-".join(k)
    b_freq_y.update({name:v})

bigram_prob = bigram_probability(b_freq, train_words_list)


In [20]:
trigram = ngrams(words,3)
t_freq = nltk.FreqDist(trigram)
t_freq_y = {}
for k,v in t_freq.items():
    name = "-".join(k)
    t_freq_y.update({name:v})
    
trigram_prob = trigram_probability(t_freq, b_freq)

# for k,v in trigram_prob.items():
#     print(k,v)

In [21]:
print(len(unigram_prob))
print(len(bigram_prob))
print(len(trigram_prob))

7606
11823
7599


In [22]:
def get_perplexity(prob_list):
    perplexity = 0
    
    for key, value in prob_list.items():
        px = value * math.log(value)
        perplexity += px
    
    perplexity = math.exp(-perplexity)
    
    return perplexity

In [23]:
unigram_perplexity = get_perplexity(unigram_prob)
# bigram_perplexity = get_perplexity(bigram_prob)
# trigram_perplexity = get_perplexity(trigram_prob)
# print(unigram_perplexity, bigram_perplexity, trigram_perplexity)
print(unigram_perplexity)

562.0336596531433


In [57]:
def laplace_smoothing(final_t_vocabulary, final_b_vocabulary, alpha, v, pw2):
    ls_trigram_prob = {}
    for i,(key, value) in enumerate(final_t_vocabulary.items()):
        p_w_n = (alpha + value) / (final_b_vocabulary[pw2[i]] + (alpha * v))
        ls_trigram_prob.update({key:p_w_n})
    
    return ls_trigram_prob
    

In [58]:
open_file = open('data/test.txt', 'r', encoding='utf-8')
test1 = open_file.read()

words = nltk.word_tokenize(test1.lower())
words = [word for word in words if word.isalpha()]
test_uni_freq = nltk.FreqDist(words)

test_trigram = ngrams(words,3)
test_t_freq = nltk.FreqDist(test_trigram)

test_t_freq_y = {}

for k,v in test_t_freq.items():
    name = "-".join(k)
    test_t_freq_y.update({name:v})

final_vocabulary = dict(train_words_list)
print(len(train_words_list))
for key, value in test_uni_freq.items():
    if key not in train_words_list.keys():
        final_vocabulary.update({key:0})
        
v = len(final_vocabulary)
print(v, len(train_words_list), len(test_uni_freq))

7606
11087 7606 6919


In [59]:
test_bigram = ngrams(words,2)
test_b_freq = nltk.FreqDist(test_bigram)
test_b_freq_y = {}


for k,v in test_b_freq.items():
    name = "-".join(k)
    test_b_freq_y.update({name:v})

final_b_vocabulary = dict(b_freq_y)
for key, value in test_b_freq_y.items():
    if key not in b_freq_y.keys():
        final_b_vocabulary.update({key:0})

final_t_vocabulary = dict(t_freq_y)
for key, value in test_t_freq_y.items():
    if key not in t_freq_y.keys():
        final_t_vocabulary.update({key:0})
print(len(final_t_vocabulary), len(t_freq), len(unigram_prob))

146049 87540 7606


In [60]:
pw2 = []
for i,(key, value) in enumerate(final_t_vocabulary.items()):
    name = "".join(key)
    p = name.split('-')
    pw = p[0] + '-' + p[1]
    pw2.append(pw)
#     print(pw2)
print(pw2[2])

scandal-in


In [66]:
def get_perplexity_2(prob_list, count_list):
    perplexity = 0
    
    for key, value in prob_list.items():
        
        px = count_list[key] * math.log2(value)
        perplexity += px
    
#     perplexity = math.exp(-perplexity)
    
    return perplexity

In [67]:
alpha = 0.1
prob_ls = laplace_smoothing(final_t_vocabulary, final_b_vocabulary, alpha, v, pw2)

ls_preplexity = get_perplexity_2(prob_ls, final_t_vocabulary )
print(ls_preplexity)

-186551.8222181604
